In [4]:
import psycopg2
import psycopg2.extras
import os
import json

POSTGRES_USER = os.environ.get('POSTGRES_USER', 'postgres')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD', '')
POSTGRES_DB = os.environ.get('POSTGRES_DB', 'objectiv')
POSTGRES_HOSTNAME = os.environ.get('POSTGRES_HOSTNAME', 'localhost')
POSTGRES_PORT = os.environ.get('POSTGRES_PORT', '5432')

connection = psycopg2.connect(user=POSTGRES_USER,
                            password=POSTGRES_PASSWORD,
                            host=POSTGRES_HOSTNAME,
                            port=POSTGRES_PORT,
                            database=POSTGRES_DB)

# This file should be mapped to a volume in the docker container,
# so changes to this file from within the container can be persisted.

with connection:
    with connection.cursor(cursor_factory=psycopg2.extras.DictCursor) as cursor:
        query = '''
                select *
                from data_with_sessions
                order by moment desc
                limit 1
            '''
        cursor.execute(query)
        row = cursor.fetchone()

if not row:
    print('No data found')
else:
    stack = row['value']
    print(json.dumps(stack, indent=4))

    print(f'Got {stack["event"]} @ {row["moment"]} session_id: {row["session_id"]} hit#: {row["session_hit_number"]}')
    for context in stack['global_contexts'] + stack['location_stack']:
        print(f'found context: {context["_context_type"]}')

No data found
